In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
pd.set_option('display.max_columns', None)
import warnings
warnings.simplefilter('ignore') # 
#import pygwalker as pyg
#from pandarallel import pandarallel

# Initialization
#pandarallel.initialize()

In [2]:
mutual_fund_holding = pd.read_csv('mutual_fund_holding_noDrops.csv')

In [6]:
mutual_fund_holding

,wficn,permno,prim_prospectus_bm,quarter,dollar_holdings,shares
0,103544.0,78736,NaN,1996Q3,6125.000000,2800.0
1,101068.0,10015,NaN,1983Q4,300000.000000,40000.0
2,101068.0,10015,NaN,1984Q1,262500.000000,40000.0
3,101068.0,10015,NaN,1984Q2,237500.000000,40000.0
4,101080.0,10015,NaN,1986Q1,490000.000000,40000.0
...,...,...,...,...,...,...
53285144,603675.0,15857,NaN,2020Q3,8489.999771,1000.0
53285145,603676.0,15857,NaN,2020Q3,16504.559555,1944.0
53285146,603798.0,15857,NaN,2020Q3,30648.899174,3610.0
53285147,604248.0,15857,NaN,2020Q3,494457.586670,58240.0


In [7]:
matched_fundno_wficn = pd.read_stata('matched_fundno_fundName_wficn_11-27-23.dta')

In [8]:
# Create a new column with 'YYYYQq' format
matched_fundno_wficn['quarter'] = matched_fundno_wficn['caldt'].dt.year.astype(str) + 'Q' + matched_fundno_wficn['caldt'].dt.quarter.astype(str)

In [9]:
matched_fundno_wficn_passive = matched_fundno_wficn[matched_fundno_wficn['index_fund_flag']=='D']

In [10]:
matched_fundno_wficn_passive = matched_fundno_wficn_passive[['wficn','quarter','index_fund_flag']].drop_duplicates()

In [11]:
mutual_fund_holding = pd.merge(mutual_fund_holding,matched_fundno_wficn_passive,on=['wficn','quarter'],how='left')

In [12]:
mutual_fund_holding['count'] = 1

In [13]:
mutual_fund_holding_bm = mutual_fund_holding[mutual_fund_holding['prim_prospectus_bm']!='no_bm'] 

In [14]:
# First, filter to include only rows with index_fund_flag == 'D'
index_funds = mutual_fund_holding_bm[mutual_fund_holding_bm['index_fund_flag'] == 'D']

# Then, group by wficn, quarter, and prim_prospectus_bm and sum the count column
permno_holdings = index_funds.groupby(['wficn', 'quarter','prim_prospectus_bm'])['count'].sum().reset_index()

In [15]:
# Calculate the median number of permno holdings for each quarter and prim_prospectus_bm
# medians = permno_holdings.groupby(['quarter', 'prim_prospectus_bm'])['count'].median().reset_index(name='median_num_permno_bm')

## Part1: the median and avg number of permno for each benchmark at each quarter

In [16]:
avg_median = permno_holdings.groupby(['quarter', 'prim_prospectus_bm'])['count'].agg(
    median_passive_num_permno_bm='median', 
    avg_passive_num_permno_bm='mean'
).reset_index()

In [17]:
avg_median

,quarter,prim_prospectus_bm,median_passive_num_permno_bm,avg_passive_num_permno_bm
0,2000Q2,MSCI EAFE NR USD,14.0,14.000000
1,2000Q2,Russell 2000 TR USD,915.0,915.000000
2,2000Q2,S&P 500 TR USD,483.0,448.000000
3,2000Q3,MSCI EAFE NR USD,14.0,14.000000
4,2000Q3,Russell 2000 TR USD,886.0,886.000000
...,...,...,...,...
721,2020Q3,Russell 2000 Growth TR USD,1088.0,1088.000000
722,2020Q3,Russell 2000 TR USD,1995.5,1835.666667
723,2020Q3,Russell 2000 Value TR USD,1439.5,1439.500000
724,2020Q3,Russell Mid Cap Growth TR USD,339.5,339.500000


In [18]:
# Permno holding of each wficn at each quarter
num_permno_holding = mutual_fund_holding_bm.groupby(['wficn', 'quarter','prim_prospectus_bm'])['count'].sum().reset_index()

In [19]:
num_permno_holding

,wficn,quarter,prim_prospectus_bm,count
0,100001.0,2003Q2,MSCI ACWI NR USD,83
1,100001.0,2003Q3,MSCI ACWI NR USD,90
2,100001.0,2003Q4,MSCI ACWI NR USD,92
3,100001.0,2004Q1,MSCI ACWI NR USD,110
4,100001.0,2004Q2,MSCI ACWI NR USD,121
...,...,...,...,...
145452,605146.0,2019Q3,S&P 500 TR USD,23
145453,605146.0,2019Q4,S&P 500 TR USD,23
145454,605146.0,2020Q1,S&P 500 TR USD,22
145455,605146.0,2020Q2,S&P 500 TR USD,9


In [20]:
num_permno_holding_passive =num_permno_holding.merge(avg_median,on=['quarter','prim_prospectus_bm'],how='left')

In [21]:
num_permno_holding_passive = num_permno_holding_passive.rename(columns={'count':'num_permno_fund'})

In [22]:
num_permno_holding_passive

,wficn,quarter,prim_prospectus_bm,num_permno_fund,median_passive_num_permno_bm,avg_passive_num_permno_bm
0,100001.0,2003Q2,MSCI ACWI NR USD,83,NaN,NaN
1,100001.0,2003Q3,MSCI ACWI NR USD,90,NaN,NaN
2,100001.0,2003Q4,MSCI ACWI NR USD,92,NaN,NaN
3,100001.0,2004Q1,MSCI ACWI NR USD,110,NaN,NaN
4,100001.0,2004Q2,MSCI ACWI NR USD,121,NaN,NaN
...,...,...,...,...,...,...
145452,605146.0,2019Q3,S&P 500 TR USD,23,476.0,498.452381
145453,605146.0,2019Q4,S&P 500 TR USD,23,242.0,280.121951
145454,605146.0,2020Q1,S&P 500 TR USD,22,471.0,453.500000
145455,605146.0,2020Q2,S&P 500 TR USD,9,500.0,499.473684


## Part2: the largest passive fund with the same benchmark

In [23]:
index_funds.dollar_holdings

616         8.966800e+04
617         3.136556e+06
623         6.796216e+04
624         1.806656e+05
627         1.375940e+04
                ...     
53285135    1.067448e+05
53285137    3.396000e+04
53285146    3.064890e+04
53285147    4.944576e+05
53285148    2.718158e+05
Name: dollar_holdings, Length: 12939877, dtype: float64

In [24]:
# Then, group by wficn, quarter, and prim_prospectus_bm and sum the count column
permno_dollar_holdings = index_funds.groupby(['wficn', 'quarter','prim_prospectus_bm'])[['count','dollar_holdings']].sum().reset_index()

In [25]:
# Define a custom function to select the row with the max dollar_holdings within each group
def get_max_dollar_holdings_row(group):
    return group.loc[group['dollar_holdings'].idxmax()]

# Apply the custom function to each group
max_dollar_holdings = permno_dollar_holdings.groupby(['quarter', 'prim_prospectus_bm']).apply(get_max_dollar_holdings_row).reset_index(drop=True)

In [26]:
max_dollar_holdings = max_dollar_holdings.rename(columns={'count':'largest_passive_num_permno_bm'}) 

In [27]:
max_dollar_holdings

,wficn,quarter,prim_prospectus_bm,largest_passive_num_permno_bm,dollar_holdings
0,500558.0,2000Q2,MSCI EAFE NR USD,14,2.889908e+07
1,102940.0,2000Q2,Russell 2000 TR USD,915,8.211967e+09
2,105866.0,2000Q2,S&P 500 TR USD,486,8.932261e+09
3,500558.0,2000Q3,MSCI EAFE NR USD,14,2.941474e+07
4,102940.0,2000Q3,Russell 2000 TR USD,886,8.212196e+09
...,...,...,...,...,...
721,500585.0,2020Q3,Russell 2000 Growth TR USD,1088,8.933285e+09
722,500109.0,2020Q3,Russell 2000 TR USD,2000,3.808704e+10
723,500594.0,2020Q3,Russell 2000 Value TR USD,1444,7.668319e+09
724,500707.0,2020Q3,Russell Mid Cap Growth TR USD,340,1.270335e+10


In [28]:
num_permno_holding_passive = num_permno_holding_passive.merge(max_dollar_holdings[['quarter','prim_prospectus_bm','largest_passive_num_permno_bm']], on=['quarter','prim_prospectus_bm'],how='left')

## The three New Benchmark Fund Sparisty Measure

In [29]:
# Calculate percentage relative to the median number of permno holdings for passive funds
num_permno_holding_passive['percent_benchmark_median_passive'] = num_permno_holding_passive['num_permno_fund'] / num_permno_holding_passive['median_passive_num_permno_bm']

# Calculate percentage relative to the average number of permno holdings for passive funds
num_permno_holding_passive['percent_benchmark_avg_passive'] = num_permno_holding_passive['num_permno_fund'] / num_permno_holding_passive['avg_passive_num_permno_bm']

# Calculate percentage relative to the largest number of permno holdings for passive funds
num_permno_holding_passive['percent_benchmark_largest_passive'] = num_permno_holding_passive['num_permno_fund'] / num_permno_holding_passive['largest_passive_num_permno_bm']


In [30]:
num_permno_holding_passive.describe().round(2)

,wficn,num_permno_fund,median_passive_num_permno_bm,avg_passive_num_permno_bm,largest_passive_num_permno_bm,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive
count,145457.00,145457.00,137655.00,137655.00,137655.0,137655.00,137655.00,137655.00
mean,264685.91,135.68,630.47,621.47,630.5,0.49,0.41,0.51
std,185914.29,252.27,460.30,442.93,452.5,2.87,1.71,2.87
min,100001.00,1.00,1.00,1.85,1.0,0.00,0.00,0.00
25%,103374.00,36.00,452.00,463.00,460.0,0.07,0.08,0.07
50%,200031.00,64.00,497.00,498.43,500.0,0.13,0.13,0.13
75%,410737.00,115.00,641.00,618.29,639.0,0.30,0.30,0.29
max,605146.00,3450.00,2000.00,1920.25,2014.0,532.00,199.50,241.50


#### Add h=0 percent_benchmark

In [31]:
mutual_fund_holding_bm

,wficn,permno,prim_prospectus_bm,quarter,dollar_holdings,shares,index_fund_flag,count
0,103544.0,78736,NaN,1996Q3,6125.000000,2800.0,NaN,1
1,101068.0,10015,NaN,1983Q4,300000.000000,40000.0,NaN,1
2,101068.0,10015,NaN,1984Q1,262500.000000,40000.0,NaN,1
3,101068.0,10015,NaN,1984Q2,237500.000000,40000.0,NaN,1
4,101080.0,10015,NaN,1986Q1,490000.000000,40000.0,NaN,1
...,...,...,...,...,...,...,...,...
53285144,603675.0,15857,NaN,2020Q3,8489.999771,1000.0,NaN,1
53285145,603676.0,15857,NaN,2020Q3,16504.559555,1944.0,NaN,1
53285146,603798.0,15857,NaN,2020Q3,30648.899174,3610.0,D,1
53285147,604248.0,15857,NaN,2020Q3,494457.586670,58240.0,D,1


In [32]:
# Step 1: Group by 'quarter', 'wficn', 'prim_prospectus_bm' to get unique permnos
unique_permnos = mutual_fund_holding_bm.groupby(['quarter', 'wficn', 'prim_prospectus_bm'])['permno'].unique().reset_index()

In [33]:
unique_permnos

,quarter,wficn,prim_prospectus_bm,permno
0,1994Q4,100968.0,Russell 2000 TR USD,"[79932, 79245, 77510, 77431, 79796, 79157, 213..."
1,1995Q4,100968.0,Russell 2000 TR USD,"[79932, 79245, 77431, 15035, 79796, 79157, 213..."
2,1996Q4,100968.0,Russell 2000 TR USD,"[79932, 77431, 80503, 15035, 21371, 10750, 758..."
3,1996Q4,105950.0,S&P 500 TR USD,"[82667, 82247, 10353, 76887, 43474, 75149, 143..."
4,1997Q4,100697.0,Russell 1000 Value TR USD,"[75034, 24221, 65138, 26518, 22947, 38893, 218..."
...,...,...,...,...
145452,2020Q3,605096.0,Russell 2000 TR USD,"[54594, 16126, 16051, 90177, 84184, 91814, 164..."
145453,2020Q3,605111.0,S&P 500 TR USD,"[57904, 66093, 20482, 13721, 75107, 79678, 892..."
145454,2020Q3,605131.0,Russell 1000 Value TR USD,"[66093, 13721, 21371, 14541, 90373, 27959, 221..."
145455,2020Q3,605137.0,Russell 2000 Value TR USD,"[54594, 90825, 17223, 47730, 77520, 10363, 128..."


In [34]:
# Aggregate to find the union of 'permno' for each 'prim_prospectus_bm' and 'quarter'
def union_permno(series):
    return set.union(*map(set, series))
# Step 2: Find the union of permnos for each benchmark and quarter
benchmark_union = unique_permnos.groupby(['quarter', 'prim_prospectus_bm'])['permno'].agg(union_permno).reset_index()

In [35]:
benchmark_union

,quarter,prim_prospectus_bm,permno
0,1994Q4,Russell 2000 TR USD,"{77696, 10757, 77190, 80135, 79245, 47248, 759..."
1,1995Q4,Russell 2000 TR USD,"{77696, 10757, 77190, 80519, 80135, 81292, 792..."
2,1996Q4,Russell 2000 TR USD,"{84096, 77696, 10757, 77190, 80519, 80135, 842..."
3,1996Q4,S&P 500 TR USD,"{81154, 77318, 77832, 82704, 80145, 10513, 821..."
4,1997Q4,Russell 1000 Value TR USD,"{49154, 21514, 21004, 48397, 75789, 48653, 594..."
...,...,...,...
876,2020Q3,Russell 2000 Growth TR USD,"{16384, 16386, 90120, 16392, 90125, 90126, 164..."
877,2020Q3,Russell 2000 TR USD,"{16384, 16386, 49154, 65541, 16392, 16399, 164..."
878,2020Q3,Russell 2000 Value TR USD,"{16384, 16386, 65541, 90120, 90121, 16392, 901..."
879,2020Q3,Russell Mid Cap Growth TR USD,"{16386, 20482, 65541, 90125, 69649, 20512, 778..."


In [36]:
# Step 3: Calculate the percentage of permno each fund holds relative to its benchmark
merged_data = pd.merge(unique_permnos, benchmark_union, on=['quarter', 'prim_prospectus_bm'], suffixes=('', '_benchmark'))

In [37]:
merged_data['percent_benchmark_0'] = merged_data.apply(lambda row: len(set(row['permno']).intersection(row['permno_benchmark'])) / len(row['permno_benchmark']), axis=1)

In [38]:
merged_data

,quarter,wficn,prim_prospectus_bm,permno,permno_benchmark,percent_benchmark_0
0,1994Q4,100968.0,Russell 2000 TR USD,"[79932, 79245, 77510, 77431, 79796, 79157, 213...","{77696, 10757, 77190, 80135, 79245, 47248, 759...",1.000000
1,1995Q4,100968.0,Russell 2000 TR USD,"[79932, 79245, 77431, 15035, 79796, 79157, 213...","{77696, 10757, 77190, 80519, 80135, 81292, 792...",1.000000
2,1996Q4,100968.0,Russell 2000 TR USD,"[79932, 77431, 80503, 15035, 21371, 10750, 758...","{84096, 77696, 10757, 77190, 80519, 80135, 842...",1.000000
3,1996Q4,105950.0,S&P 500 TR USD,"[82667, 82247, 10353, 76887, 43474, 75149, 143...","{81154, 77318, 77832, 82704, 80145, 10513, 821...",1.000000
4,1997Q4,100697.0,Russell 1000 Value TR USD,"[75034, 24221, 65138, 26518, 22947, 38893, 218...","{49154, 21514, 21004, 48397, 75789, 48653, 594...",0.361538
...,...,...,...,...,...,...
145452,2020Q3,605096.0,Russell 2000 TR USD,"[54594, 16126, 16051, 90177, 84184, 91814, 164...","{16384, 16386, 49154, 65541, 16392, 16399, 164...",0.187059
145453,2020Q3,605111.0,S&P 500 TR USD,"[57904, 66093, 20482, 13721, 75107, 79678, 892...","{16384, 49154, 16386, 65541, 16392, 32803, 164...",0.102446
145454,2020Q3,605131.0,Russell 1000 Value TR USD,"[66093, 13721, 21371, 14541, 90373, 27959, 221...","{20482, 49154, 65541, 90120, 69649, 12308, 123...",0.019275
145455,2020Q3,605137.0,Russell 2000 Value TR USD,"[54594, 90825, 17223, 47730, 77520, 10363, 128...","{16384, 16386, 65541, 90120, 90121, 16392, 901...",0.329507


In [39]:
num_permno_holding_passive = num_permno_holding_passive.merge(merged_data[['quarter','wficn','percent_benchmark_0']],on=['quarter','wficn'],how='left')

In [40]:
num_permno_holding_passive

,wficn,quarter,prim_prospectus_bm,num_permno_fund,median_passive_num_permno_bm,avg_passive_num_permno_bm,largest_passive_num_permno_bm,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive,percent_benchmark_0
0,100001.0,2003Q2,MSCI ACWI NR USD,83,NaN,NaN,NaN,NaN,NaN,NaN,0.082505
1,100001.0,2003Q3,MSCI ACWI NR USD,90,NaN,NaN,NaN,NaN,NaN,NaN,0.087719
2,100001.0,2003Q4,MSCI ACWI NR USD,92,NaN,NaN,NaN,NaN,NaN,NaN,0.083789
3,100001.0,2004Q1,MSCI ACWI NR USD,110,NaN,NaN,NaN,NaN,NaN,NaN,0.096661
4,100001.0,2004Q2,MSCI ACWI NR USD,121,NaN,NaN,NaN,NaN,NaN,NaN,0.091667
...,...,...,...,...,...,...,...,...,...,...,...
145452,605146.0,2019Q3,S&P 500 TR USD,23,476.0,498.452381,482.0,0.048319,0.046143,0.047718,0.006694
145453,605146.0,2019Q4,S&P 500 TR USD,23,242.0,280.121951,244.0,0.095041,0.082107,0.094262,0.007556
145454,605146.0,2020Q1,S&P 500 TR USD,22,471.0,453.500000,474.0,0.046709,0.048512,0.046414,0.006513
145455,605146.0,2020Q2,S&P 500 TR USD,9,500.0,499.473684,504.0,0.018000,0.018019,0.017857,0.002216


In [41]:
num_permno_holding_passive.describe().round(2)

,wficn,num_permno_fund,median_passive_num_permno_bm,avg_passive_num_permno_bm,largest_passive_num_permno_bm,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive,percent_benchmark_0
count,145457.00,145457.00,137655.00,137655.00,137655.0,137655.00,137655.00,137655.00,145457.00
mean,264685.91,135.68,630.47,621.47,630.5,0.49,0.41,0.51,0.06
std,185914.29,252.27,460.30,442.93,452.5,2.87,1.71,2.87,0.09
min,100001.00,1.00,1.00,1.85,1.0,0.00,0.00,0.00,0.00
25%,103374.00,36.00,452.00,463.00,460.0,0.07,0.08,0.07,0.01
50%,200031.00,64.00,497.00,498.43,500.0,0.13,0.13,0.13,0.03
75%,410737.00,115.00,641.00,618.29,639.0,0.30,0.30,0.29,0.06
max,605146.00,3450.00,2000.00,1920.25,2014.0,532.00,199.50,241.50,1.00


In [42]:
fund_sparisty = pd.read_pickle('2024_04_11_fund_sparsity.pickle')

In [43]:
columns_list = [
    'wficn',
    'quarter',
    'percent_benchmark_median_passive',
    'percent_benchmark_avg_passive',
    'percent_benchmark_largest_passive',
    'percent_benchmark_0'
]

In [44]:
fund_sparisty = fund_sparisty.merge(num_permno_holding_passive[columns_list],on=['quarter','wficn'],how='left')

In [45]:
fund_sparisty

,wficn,quarter,prim_prospectus_bm,percent_within_3,if_past_quarter_missing_3,percent_within_7,if_past_quarter_missing_7,percent_within_11,if_past_quarter_missing_11,percent_within_15,if_past_quarter_missing_15,percent_within_19,if_past_quarter_missing_19,percent_benchmark_3,percent_benchmark_7,percent_benchmark_11,percent_benchmark_15,percent_benchmark_19,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive,percent_benchmark_0
0,100001.0,1990Q3,NaN,1.000000,1,1.000000,1,1.000000,1,1.000000,1,1.000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100001.0,1990Q4,NaN,1.000000,1,1.000000,1,1.000000,1,1.000000,1,1.000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100001.0,1991Q1,NaN,0.982759,1,0.982759,1,0.982759,1,0.982759,1,0.982759,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100001.0,1991Q2,NaN,0.950000,0,0.950000,1,0.950000,1,0.950000,1,0.950000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100001.0,1991Q3,NaN,0.915254,0,0.915254,1,0.915254,1,0.915254,1,0.915254,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425641,605153.0,2019Q3,NaN,0.909271,0,0.874952,1,0.874952,1,0.874952,1,0.874952,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425642,605153.0,2019Q4,NaN,0.615907,0,0.597222,1,0.597222,1,0.597222,1,0.597222,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425643,605153.0,2020Q1,NaN,0.625335,0,0.606083,1,0.606083,1,0.606083,1,0.606083,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425644,605153.0,2020Q2,NaN,0.939383,0,0.901054,0,0.901054,1,0.901054,1,0.901054,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
columns_list = [
    "percent_benchmark_0",
    "percent_benchmark_3",
    "percent_benchmark_7",
    "percent_benchmark_11",
    "percent_benchmark_15",
    "percent_benchmark_19",
    "percent_benchmark_median_passive",
    "percent_benchmark_avg_passive",
    "percent_benchmark_largest_passive"
]
fund_sparisty[columns_list].describe().round(2)

,percent_benchmark_0,percent_benchmark_3,percent_benchmark_7,percent_benchmark_11,percent_benchmark_15,percent_benchmark_19,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive
count,145457.00,145457.00,145457.00,145457.00,145457.00,145457.00,137655.00,137655.00,137655.00
mean,0.06,0.05,0.05,0.04,0.04,0.04,0.49,0.41,0.51
std,0.09,0.08,0.07,0.07,0.07,0.07,2.87,1.71,2.87
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.01,0.01,0.01,0.01,0.01,0.01,0.07,0.08,0.07
50%,0.03,0.03,0.02,0.02,0.02,0.02,0.13,0.13,0.13
75%,0.06,0.05,0.05,0.04,0.04,0.04,0.30,0.30,0.29
max,1.00,1.00,1.00,1.00,1.00,1.00,532.00,199.50,241.50


In [52]:
fund_sparisty = fund_sparisty[['wficn', 'quarter', 'prim_prospectus_bm', 'percent_within_3',
       'if_past_quarter_missing_3', 'percent_within_7',
       'if_past_quarter_missing_7', 'percent_within_11',
       'if_past_quarter_missing_11', 'percent_within_15',
       'if_past_quarter_missing_15', 'percent_within_19',
       'if_past_quarter_missing_19', 'percent_benchmark_0','percent_benchmark_3',
       'percent_benchmark_7', 'percent_benchmark_11', 'percent_benchmark_15',
       'percent_benchmark_19', 'percent_benchmark_median_passive',
       'percent_benchmark_avg_passive', 'percent_benchmark_largest_passive']]

In [53]:
fund_sparisty

,wficn,quarter,prim_prospectus_bm,percent_within_3,if_past_quarter_missing_3,percent_within_7,if_past_quarter_missing_7,percent_within_11,if_past_quarter_missing_11,percent_within_15,if_past_quarter_missing_15,percent_within_19,if_past_quarter_missing_19,percent_benchmark_0,percent_benchmark_3,percent_benchmark_7,percent_benchmark_11,percent_benchmark_15,percent_benchmark_19,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive
0,100001.0,1990Q3,NaN,1.000000,1,1.000000,1,1.000000,1,1.000000,1,1.000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100001.0,1990Q4,NaN,1.000000,1,1.000000,1,1.000000,1,1.000000,1,1.000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100001.0,1991Q1,NaN,0.982759,1,0.982759,1,0.982759,1,0.982759,1,0.982759,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100001.0,1991Q2,NaN,0.950000,0,0.950000,1,0.950000,1,0.950000,1,0.950000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100001.0,1991Q3,NaN,0.915254,0,0.915254,1,0.915254,1,0.915254,1,0.915254,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425641,605153.0,2019Q3,NaN,0.909271,0,0.874952,1,0.874952,1,0.874952,1,0.874952,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425642,605153.0,2019Q4,NaN,0.615907,0,0.597222,1,0.597222,1,0.597222,1,0.597222,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425643,605153.0,2020Q1,NaN,0.625335,0,0.606083,1,0.606083,1,0.606083,1,0.606083,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425644,605153.0,2020Q2,NaN,0.939383,0,0.901054,0,0.901054,1,0.901054,1,0.901054,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
fund_sparisty.to_csv('2024_04_11_fund_sparisty.csv',index=False)